# Snore Detection from Audio — Jupyter Notebook

This notebook shows an end-to-end pipeline for building a snore detection classifier from audio files.
It extracts features (mel-spectrograms) and trains a small CNN using TensorFlow / Keras.

**Important:** Adjust the labeling logic inside the notebook if your dataset uses different filenames or a CSV for labels.


In [ ]:
# Install dependencies if necessary
!pip install --quiet librosa soundfile tensorflow scikit-learn matplotlib tqdm


In [ ]:
# Setup: extract archive.zip if present
import os, zipfile
ARCHIVE='/mnt/data/archive.zip'
EXTRACT_DIR='/mnt/data/extracted_audio'
if os.path.exists(ARCHIVE):
    os.makedirs(EXTRACT_DIR, exist_ok=True)
    with zipfile.ZipFile(ARCHIVE,'r') as z:
        z.extractall(EXTRACT_DIR)
    print('Extracted to', EXTRACT_DIR)
else:
    print('No archive found at', ARCHIVE)


In [ ]:
# Inspect audio files
from glob import glob
audio_paths = glob('/mnt/data/extracted_audio/**', recursive=True)
audio_paths = [p for p in audio_paths if p.lower().endswith(('.wav','.flac','.mp3','.m4a','.ogg'))]
print('Found', len(audio_paths), 'audio files')
for p in audio_paths[:10]:
    print(p)


In [ ]:
# Simple heuristic labeling function - edit if needed
def get_label_from_path(path):
    n = os.path.basename(path).lower()
    if 'snore' in n or 'snores' in n or 'snoring' in n:
        return 1
    lower = path.lower()
    if '/snore/' in lower or '\\snore\\' in lower:
        return 1
    return 0

# quick distribution
labels = [get_label_from_path(p) for p in audio_paths]
from collections import Counter
print('Label counts (heuristic):', Counter(labels))


In [ ]:
# Feature extraction (mel spectrograms)
import librosa, numpy as np
from tqdm import tqdm
SR=22050
DURATION=4.0
SAMPLES=int(SR*DURATION)
N_MELS=64
HOP_LENGTH=512

def load_audio(path):
    x,_ = librosa.load(path,sr=SR,mono=True,duration=DURATION)
    if len(x) < SAMPLES:
        x = np.pad(x,(0,SAMPLES-len(x)))
    else:
        x = x[:SAMPLES]
    return x

def extract_mel(path):
    x = load_audio(path)
    mel = librosa.feature.melspectrogram(y=x, sr=SR, n_mels=N_MELS, hop_length=HOP_LENGTH)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    mel_db = (mel_db - mel_db.mean())/(mel_db.std()+1e-6)
    return mel_db.astype(np.float32)

X_list=[]
y_list=[]
for p in tqdm(audio_paths):
    try:
        X_list.append(extract_mel(p))
        y_list.append(get_label_from_path(p))
    except Exception as e:
        print('skip',p,e)

X = np.array(X_list)
y = np.array(y_list)
print('X',X.shape,'y',y.shape)


In [ ]:
# Prepare data for Keras
X = X[..., np.newaxis]
print('Input shape:', X.shape)
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X,y,test_size=0.30,random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp,y_temp,test_size=0.50,random_state=42, stratify=y_temp)
print('Train',X_train.shape,'Val',X_val.shape,'Test',X_test.shape)


In [ ]:
# Build a small CNN (Keras)
import tensorflow as tf
from tensorflow.keras import layers, models
input_shape = X_train.shape[1:]
model = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(16,(3,3),activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),

    layers.Conv2D(32,(3,3),activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),

    layers.Conv2D(64,(3,3),activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.3),

    layers.GlobalAveragePooling2D(),
    layers.Dense(64,activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])
model.summary()


In [ ]:
# Train (adjust EPOCHS/BATCH_SIZE for your environment)
EPOCHS=12
BATCH_SIZE=16
callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)]
history = model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=callbacks)


In [ ]:
# Evaluate and save
res = model.evaluate(X_test,y_test)
print('Test results:', res)
model.save('/mnt/data/snore_detector_model.h5')
print('Model saved to /mnt/data/snore_detector_model.h5')


In [ ]:
# Quick predict helper
from tensorflow.keras.models import load_model
m = load_model('/mnt/data/snore_detector_model.h5')

def predict_snore(path):
    mel = extract_mel(path)
    mel = mel[np.newaxis,...,np.newaxis]
    p = m.predict(mel)[0,0]
    return p

if len(audio_paths)>0:
    print(audio_paths[0],'->',predict_snore(audio_paths[0]))
